Если начнут вылезать ошибки, попробуйте загрузить на гугл и посмотреть через гугл коллаб

# Необходимые импорты, доки и функции, просто запусти

In [ ]:
import pandas as pd
import datetime
import requests
import gdown
import re

# Ваш API-ключ
API_KEY = 'd5831c09-afb1-4a6b-95af-2e0121862246'

In [ ]:
!gdown 1qX_mCo8qfUqyFKazAF-hs-gCa6kP-yT8
!gdown 1cIzqqWPXSawFcdy4c963GY1xOi37bDVk
!gdown 1kExtP7kLKToJ65YSoK76fnwoLyoF2Jrv
!gdown 1M31m-Y7AdhOQ3IgorGFg1Pw6eNJvrvc7
df_station = pd.read_csv('/content/code_of_station')
df_type_station = pd.read_csv('/content/type_station')
df_piter = pd.read_csv('/content/piter')
df_moscwa = pd.read_csv('/content/moscwa')
df_station = df_station.rename(columns={"Регион": "Город", "страна": "Регион", "код региона": "код города", "код страны": "код региона"})

Downloading...
From: https://drive.google.com/uc?id=1qX_mCo8qfUqyFKazAF-hs-gCa6kP-yT8
To: /content/code_of_station
100% 9.10M/9.10M [00:00<00:00, 49.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1cIzqqWPXSawFcdy4c963GY1xOi37bDVk
To: /content/type_station
100% 200k/200k [00:00<00:00, 27.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1kExtP7kLKToJ65YSoK76fnwoLyoF2Jrv
To: /content/piter
100% 11.2k/11.2k [00:00<00:00, 28.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1M31m-Y7AdhOQ3IgorGFg1Pw6eNJvrvc7
To: /content/moscwa
100% 15.1k/15.1k [00:00<00:00, 38.5MB/s]


Все функции которые есть у меня:

In [ ]:
###Ищем коды для наших городов
### на вход: строка с названием города
### на выход: строка с кодом
def gorod2cod_goroda(gorod):
  result = df_station.query(f"Город == '{gorod}'")
  return result['код города'].values[0]

###Ищем пандас таблицу маршрутов по двум точкам
### на вход: два города в виде кода города
### на выход: таблица пандас с описанием маршрутов
def goroda2tabl_of_route(fr, to):
  # URL для доступа к API Яндекс Расписания
  url = 'https://api.rasp.yandex.net/v3.0/search/'

  # Параметры запроса
  params = {
      'from': fr,
      'to': to,
      'date': date,
      'apikey': API_KEY
  }

  # Выполнить GET-запрос
  response = requests.get(url, params=params)

  # Проверка статуса ответа
  if response.status_code == 200:
      data = response.json()
      # print(data)  # Выводим данные расписания
  else:
      data = None
  route_list = []
  for i, num in enumerate(data['segments']):
    thread = data['segments'][i]['thread']['number']
    transport_type = data['segments'][i]['thread']['transport_type']
    fr = data['segments'][i]['from']['title']
    to = data['segments'][i]['to']['title']
    time = data['segments'][i]['duration']
    departure =  data['segments'][i]['departure']
    arrival = data['segments'][i]['arrival']
    route_list.append([thread, fr, to, transport_type, departure, arrival, time])
  df_res = pd.DataFrame(route_list, columns=['номер маршрута', "начальная точка", "конечная точка", "вид транспорта", 'отправления', "прибытие", "время в секундах"])
  return df_res


###Сортируем таблицу с маршрутами
### на вход: не отсортированный массив
### на выход: отсортированный массив
def sort_route(route):
  return route.sort_values('время в секундах')


###Ищет по городу все города в которые можно проехать одним транспортом
###на взод: один город и возможно потом добавлю фильтр по остановкам
###на выход список всех городов


# Условия для вункции поиска городов
# Если станций больше 35 а поездов больше 3 то автобусы нафиг
# Если поездов 0 а станций больше 25  идём с шагом 2
# Если станций больше 30 и меньше 55 берём автобуы с шагом 2
# Если станций больше 55 и меньше 90 берём автобуы с шагом 3
# Если станций больше 90  и меньше 90 берём автобуы с шагом 4
# Если станций больше 120 и меньше 150 берём автобуы с шагом 5
# Если станций больше 150 берём автобуы с шагом 6
# для Москвы и Питера делаем отдельно



def find_all_goroda(us_gorod, date):
  if us_gorod == 'Москва':
    return df_moscwa['Направления'].values.tolist()
  elif us_gorod == 'Санкт-Петербург':
    return df_piter['Направления'].values.tolist()
  else:
    col_st = df_type_station.query(f"Город == '{us_gorod}'")['Все станции'].values[0]
    result = df_station.query(f"Город == '{us_gorod}'")
    result_no_bus = result.loc[result['тип транспорта'] != 'bus']['код станции'].values.tolist()
    result_bus = result.loc[result['тип транспорта'] == 'bus']['код станции'].values.tolist()


    if col_st <= 30:
      for b_st in range(0, len(result_bus), 1):
        result_no_bus.append(result_bus[b_st])
    elif col_st > 30 and col_st <= 80:
      for b_st in range(0, len(result_bus), 1):
        result_no_bus.append(result_bus[b_st])
    elif col_st > 80 and col_st <= 130:
      for b_st in range(0, len(result_bus), 2):
        result_no_bus.append(result_bus[b_st])
    elif col_st > 130:
      for b_st in range(0, len(result_bus), 2):
        result_no_bus.append(result_bus[b_st])


    # URL для доступа к API Яндекс Расписания
    url = 'https://api.rasp.yandex.net/v3.0/schedule'

    station_uid = []

    direct_state = []

    #Строим цикл для запроса всех путей
    for i, code in enumerate(result_no_bus):
      # print(code)
      params = {
        'station': code,
        'date':date,
        'event':'departure',
        # 'system':'yandex',
        'apikey': API_KEY
      }
      response = requests.get(url, params=params)
      # Проверка статуса ответа
      if response.status_code == 200:
          data = response.json()
          for thr in data['schedule']:
            if set(process_string(thr['thread']['title'])) not in direct_state:
              direct_state.append(set(process_string(thr['thread']['title'])))
              station_uid.append([code, thr['thread']['uid'], thr['departure']])

      else:
          continue
    direct_state = set([re.sub(r",.*", "", add_space(a)) for b in direct_state for a in b])
    return direct_state


###Функция находит маршруты от одних городов в другие и выводит списком
###на вход: два города, два списка городов или город и список. города представляем ввиде кода
###на выход: пандас таблица возможных маршрутов межгородних
def goroda2goroda(goroda_fr, goroda_to):
  # URL для доступа к API Яндекс Расписания
  url = 'https://api.rasp.yandex.net/v3.0/search/'

  if type(goroda_fr) != type(['a','b']):
    goroda_f = []
    goroda_f.append(goroda_fr)
    goroda_fr = goroda_f

  if type(goroda_to) != type(['a','b']):
    goroda_t = []
    goroda_t.append(goroda_to)
    goroda_to = goroda_t

  route_list = []

  for a in goroda_fr:
    for b in goroda_to:
      # Параметры запроса
      params = {
          'from': a,
          'to': b,
          'date': date,
          'apikey': API_KEY
      }

      # Выполнить GET-запрос
      response = requests.get(url, params=params)

      # Проверка статуса ответа
      if response.status_code == 200:
          data = response.json()
          for i, num in enumerate(data['segments']):
            thread = data['segments'][i]['thread']['number']
            transport_type = data['segments'][i]['thread']['transport_type']
            fro = data['segments'][i]['from']['title']
            too = data['segments'][i]['to']['title']
            time = data['segments'][i]['duration']
            departure =  data['segments'][i]['departure']
            arrival = data['segments'][i]['arrival']
            route_list.append([thread, fro, too, transport_type, departure, arrival, time])
      else:
          continue

    return pd.DataFrame(route_list, columns=['номер маршрута', "начальная точка", "конечная точка", "вид транспорта", 'отправления', "прибытие", "время в секундах"])

def goroda2goroda_2_days(goroda_fr, goroda_to):
  # URL для доступа к API Яндекс Расписания
  url = 'https://api.rasp.yandex.net/v3.0/search/'

  if type(goroda_fr) != type(['a','b']):
    goroda_f = []
    goroda_f.append(goroda_fr)
    goroda_fr = goroda_f

  if type(goroda_to) != type(['a','b']):
    goroda_t = []
    goroda_t.append(goroda_to)
    goroda_to = goroda_t

  date2 = next_day(date)

  date3 = next_day(date2)

  route_list = []

  for a in goroda_fr:
    for b in goroda_to:
      for d in[date, date2, date3]:
        # Параметры запроса
        params = {
            'from': a,
            'to': b,
            'date': d,
            'apikey': API_KEY
        }

        # Выполнить GET-запрос
        response = requests.get(url, params=params)

        # Проверка статуса ответа
        if response.status_code == 200:
            data = response.json()
            for i, num in enumerate(data['segments']):
              thread = data['segments'][i]['thread']['number']
              transport_type = data['segments'][i]['thread']['transport_type']
              fro = data['segments'][i]['from']['title']
              too = data['segments'][i]['to']['title']
              time = data['segments'][i]['duration']
              departure =  data['segments'][i]['departure']
              arrival = data['segments'][i]['arrival']
              route_list.append([thread, fro, too, transport_type, departure, arrival, time])
        else:
            continue

  return pd.DataFrame(route_list, columns=['номер маршрута', "начальная точка", "конечная точка", "вид транспорта", 'отправления', "прибытие", "время в секундах"])




def route_df2date2state(rout_from_start):
  #Меняем на датовый тип данных
  rout_from_start['отправления'] = pd.to_datetime(rout_from_start['отправления'], utc=True)
  rout_from_start['прибытие'] = pd.to_datetime(rout_from_start['прибытие'], utc=True)

  # Добавляем города в которых находятся станции
  table = df_station[['Станция', 'Город']]

  # Переименование столбцов для объединения
  table = table.rename(columns={'Станция': 'начальная точка'})
  # Объединение таблиц по ключу
  merged_table = pd.merge(rout_from_start, table, on='начальная точка', how='left')
  merged_table = merged_table.rename(columns={'Город': 'начальная точка: Город'})

  #Тоже самое для прибытия
  # Переименование столбцов для объединения
  table = df_station[['Станция', 'Город']]
  table = table.rename(columns={'Станция': 'конечная точка'})
  # Объединение таблиц по ключу
  merged_table = pd.merge(merged_table, table, on='конечная точка', how='left')
  merged_table = merged_table.rename(columns={'Город': 'конечная точка: Город'})
  return merged_table


def complex_route(table_1, table_2):
  # Объединение по ключу
  table1 = table_1.rename(columns={'конечная точка: Город': 'Город пересадки'})
  table2 = table_2.rename(columns={'начальная точка: Город': 'Город пересадки'})
  merged = pd.merge(table1, table2, on='Город пересадки', suffixes=('_до_пересадки', '_после_пересадки_df2'))

  # Фильтрация по условиям
  result = merged[(merged['прибытие_до_пересадки'] < merged['отправления_после_пересадки_df2'])]

  return result

def next_day(a):
  mon = {'01': 31,'02':28, '03':31,'04':30, '05':31,'06':30, '07':31, '08':31, '09':30, '10':31, '11':30, '12':31}
  if int(a[-2:]) < mon[a[5:7]]:
    b = a[:-2] + str(int(a[-2:])+1)
  else:
    if a[5:7] == '09' or a[5:7] == '10' or a[5:7] == '11':
      b = a[0:4] + '-' + str(int(a[5:7])+1) +'-01'
    elif a[5:7] == '12' :
      b = str(int(a[0:4])+1) + '-01' + '-01'
    else:
      b = a[0:4] + '-0' + str(int(a[5:7])+1) +'-01'
  return b

def process_string(input_string):
    # Разбиваем строку по символу тире
    parts = input_string.split('—')
    # Удаляем часть в скобках из каждого элемента
    result = [re.sub(r'\s*\(.*?\)\s*', '', part) for part in parts]
    result = [part.replace('(', '').replace(')', '').replace('.', '') for part in result]
    res = []
    for resu in result:
      resu = resu.lstrip().rstrip()
      res.append(resu)
    return res


def add_space(text):
    result = ''
    for i in range(len(text)):
        if i > 0 and text[i].isupper() and text[i-1].islower():
            result += ' '
        result += text[i]
    return result

#Построй свой самый лучший маршрут

Введите Город отправление и прибытия а также дату начала маршрута

In [ ]:
# Обязательные параметры
start_state = 'Ярославль' # Запишите в кавычках название города начала маршрута в именительном падеже с заглавной буквы
end_state = 'Тверь'       # Запишите в кавычках название города конца маршрута в именительном падеже с заглавной буквы
date = '2025-03-18'         # Запишите в кавычках дату начала маршрута в формате 'ГГГГ-ММ-ДД'

#Расчёты, просто нажми. Может занять несколько минут.

In [ ]:
#1. достаём коды городов
code_start = gorod2cod_goroda(start_state)
code_end = gorod2cod_goroda(end_state)

#2. Дальше находим прямые маршруты
direct_route = goroda2tabl_of_route(code_start, code_end)

#3. Находит маршруты с одной пересадкой, для этого
#а. Находим города в которые можно приехать из начального города
all_direct_start = find_all_goroda(start_state, date)
all_direct_end = find_all_goroda(end_state, date)

all_state = []
if len(all_direct_start) < 10 and len(all_direct_end) >= 10:
  all_state = all_direct_start
elif len(all_direct_start) < 10 and len(all_direct_end) < 10:
  for i in all_direct_start:
    all_state.append(i)
  for i in all_direct_end:
    all_state.append(i)
elif len(all_direct_end) < 10 and len(all_direct_start) >= 10:
  all_state = all_direct_end
else:
  for i in all_direct_start:
    if i in all_direct_end and i!=start_state and i!=end_state and i in df_station['Город'].values.tolist():
      all_state.append(i)

#б. Строим маршруты от старта до этих городов и от этих городов до финиша
all_state_code = []
for i in set(all_state):
  all_state_code.append(gorod2cod_goroda(i))

rout_from_start = goroda2goroda(code_start, all_state_code)
rout_to_end = goroda2goroda_2_days(all_state_code, code_end)


#Теперь нужно привести таблици к виду где будет добавлен город старта и финиша,
#а не только станция и дату преобразовать во время, тоже нужна функция
new_rout_from_start = route_df2date2state(rout_from_start)
new_rout_to_end = route_df2date2state(rout_to_end)

#Соединяем две таблицы и получаем сложный маршрут
table_with_complex_route = complex_route(new_rout_from_start, new_rout_to_end)

#Запусти, чтобы увидеть возможные маршруты

Таблица маршрута без пересадки

In [ ]:
direct_route

,номер маршрута,начальная точка,конечная точка,вид транспорта,отправления,прибытие,время в секундах


Таблица маршрутов с одной пересадкой

In [ ]:
table_with_complex_route

,номер маршрута_до_пересадки,начальная точка_до_пересадки,конечная точка_до_пересадки,вид транспорта_до_пересадки,отправления_до_пересадки,прибытие_до_пересадки,время в секундах_до_пересадки,начальная точка: Город,Город пересадки,номер маршрута_после_пересадки_df2,начальная точка_после_пересадки_df2,конечная точка_после_пересадки_df2,вид транспорта_после_пересадки_df2,отправления_после_пересадки_df2,прибытие_после_пересадки_df2,время в секундах_после_пересадки_df2,конечная точка: Город
20,069Ь,Ярославль-Главный,Москва (Ярославский вокзал),train,2025-03-17 21:33:00+00:00,2025-03-18 01:56:00+00:00,15780.0,Ярославль,Москва,6702,Москва (Ленинградский вокзал),Тверь,suburban,2025-03-18 02:04:00+00:00,2025-03-18 04:49:00+00:00,9900.0,Тверь
21,069Ь,Ярославль-Главный,Москва (Ярославский вокзал),train,2025-03-17 21:33:00+00:00,2025-03-18 01:56:00+00:00,15780.0,Ярославль,Москва,6702,Москва (Ленинградский вокзал),Тверь,suburban,2025-03-18 02:04:00+00:00,2025-03-18 04:49:00+00:00,9900.0,Тверь
22,069Ь,Ярославль-Главный,Москва (Ярославский вокзал),train,2025-03-17 21:33:00+00:00,2025-03-18 01:56:00+00:00,15780.0,Ярославль,Москва,7924,Москва (Ленинградский вокзал),Тверь,suburban,2025-03-18 03:41:00+00:00,2025-03-18 05:53:00+00:00,7920.0,Тверь
23,069Ь,Ярославль-Главный,Москва (Ярославский вокзал),train,2025-03-17 21:33:00+00:00,2025-03-18 01:56:00+00:00,15780.0,Ярославль,Москва,7924,Москва (Ленинградский вокзал),Тверь,suburban,2025-03-18 03:41:00+00:00,2025-03-18 05:53:00+00:00,7920.0,Тверь
24,069Ь,Ярославль-Главный,Москва (Ярославский вокзал),train,2025-03-17 21:33:00+00:00,2025-03-18 01:56:00+00:00,15780.0,Ярославль,Москва,756А,Москва (Ленинградский вокзал),Тверь,train,2025-03-18 04:10:00+00:00,2025-03-18 05:16:00+00:00,3960.0,Тверь
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11583,RT 640,Туношна,Казань,plane,2025-03-18 12:30:00+00:00,2025-03-18 13:40:00+00:00,4200.0,Туношна,Казань,133Г,Казань-Пасс.,Тверь,train,2025-03-19 13:10:00+00:00,2025-03-20 04:55:00+00:00,56700.0,Тверь
11584,RT 640,Туношна,Казань,plane,2025-03-18 12:30:00+00:00,2025-03-18 13:40:00+00:00,4200.0,Туношна,Казань,133Г,Казань-Пасс.,Тверь,train,2025-03-20 13:10:00+00:00,2025-03-21 05:23:00+00:00,58380.0,Тверь
11585,RT 640,Туношна,Казань,plane,2025-03-18 12:30:00+00:00,2025-03-18 13:40:00+00:00,4200.0,Туношна,Казань,133Г,Казань-Пасс.,Тверь,train,2025-03-20 13:10:00+00:00,2025-03-21 05:23:00+00:00,58380.0,Тверь
11594,079Я,Ярославль-Главный,Адлер,train,2025-03-18 18:25:00+00:00,2025-03-20 10:15:00+00:00,143400.0,Ярославль,Адлер,036С,Адлер,Тверь,train,2025-03-20 14:44:00+00:00,2025-03-22 01:09:00+00:00,123900.0,Тверь


Ранжирование непрямых маршрутов (чем меньше рейтинг - тем более подходящий маршрут)

In [ ]:
import pandas as pd
import numpy as np

def filter_routes(df, exclude_transport=None, min_transfer_time=None, max_transfer_time=None, preferred_time_range=None, weight_time=0.5, weight_price=0.5):
    """
    Фильтрация маршрутов по заданным параметрам.
    """
    df = df.drop_duplicates(subset=['начальная точка_до_пересадки', 'вид транспорта_до_пересадки', 'отправления_до_пересадки', 'прибытие_до_пересадки'])

    if exclude_transport:
        df = df[~df['вид транспорта_до_пересадки'].isin(exclude_transport)]
        df = df[~df['вид транспорта_после_пересадки_df2'].isin(exclude_transport)]

    # Вычисляем время пересадки в секундах
    df['время пересадки'] = (pd.to_datetime(df['отправления_после_пересадки_df2']) - pd.to_datetime(df['прибытие_до_пересадки'])).dt.total_seconds()

    # Корректное ограничение минимального времени пересадки
    if min_transfer_time is None:
        df['мин_время_пересадки'] = df.apply(
            lambda row: 3 * 3600 if row['вид транспорта_до_пересадки'] == 'plane' or row['вид транспорта_после_пересадки_df2'] == 'plane' else 2 * 3600,
            axis=1
        )
    else:
        df['мин_время_пересадки'] = df.apply(
            lambda row: max(min_transfer_time, 3 * 3600) if row['вид транспорта_до_пересадки'] == 'plane' or row['вид транспорта_после_пересадки_df2'] == 'plane' else min_transfer_time,
            axis=1
        )

    if max_transfer_time is None:
        max_transfer_time = 6 * 3600

    df = df[(df['время пересадки'] >= df['мин_время_пересадки']) & (df['время пересадки'] <= max_transfer_time)]

    price_map = {
        'plane': 5000,
        'train': 300,
        'bus': 150,
        'suburban': 200,
    }

    df['цена первого маршрута'] = (df['время в секундах_до_пересадки'] / 3600 * df['вид транспорта_до_пересадки'].map(price_map)).astype(int)
    df['цена второго маршрута'] = (df['время в секундах_после_пересадки_df2'] / 3600 * df['вид транспорта_после_пересадки_df2'].map(price_map)).astype(int)
    df['общая_цена'] = df['цена первого маршрута'] + df['цена второго маршрута']

    df['длительность первого маршрута'] = df['время в секундах_до_пересадки'].apply(lambda x: f"{int(x // 3600)}:{int((x % 3600) // 60):02d}")
    df['длительность второго маршрута'] = df['время в секундах_после_пересадки_df2'].apply(lambda x: f"{int(x // 3600)}:{int((x % 3600) // 60):02d}")
    df['время пересадки'] = df['время пересадки'].apply(lambda x: f"{int(x // 3600)}:{int((x % 3600) // 60):02d}")

    df['общее_время'] = df['время в секундах_до_пересадки'] + df['время пересадки'].apply(lambda x: int(x.split(':')[0]) * 3600 + int(x.split(':')[1]) * 60) + df['время в секундах_после_пересадки_df2']
    df['общее_время'] = df['общее_время'].apply(lambda x: f"{int(x // 3600)}:{int((x % 3600) // 60):02d}")

    df = df.drop(columns=['время в секундах_до_пересадки', 'время в секундах_после_пересадки_df2', 'мин_время_пересадки'])

    if preferred_time_range:
        start_hour, end_hour = preferred_time_range
        if start_hour > end_hour:
            df = df[(df['отправления_до_пересадки'].dt.hour >= start_hour) | (df['отправления_до_пересадки'].dt.hour <= end_hour)]
        else:
            df = df[(df['отправления_до_пересадки'].dt.hour >= start_hour) & (df['отправления_до_пересадки'].dt.hour <= end_hour)]

    df['рейтинг'] = weight_time * (df['общее_время'].apply(lambda x: int(x.split(':')[0]) * 60 + int(x.split(':')[1])) / df['общее_время'].apply(lambda x: int(x.split(':')[0]) * 60 + int(x.split(':')[1])).max()) + weight_price * (df['общая_цена'] / df['общая_цена'].max())
    df = df.sort_values(by='рейтинг', ascending=True)

    return df

# Пример вызова
df_filtered = filter_routes(
    table_with_complex_route,
    exclude_transport=['plane'],
    min_transfer_time=2 * 3600,
    max_transfer_time=6 * 3600,
    preferred_time_range=(22, 8),
    weight_time=0.7,
    weight_price=0.3
)
print(df_filtered)


     номер маршрута_до_пересадки начальная точка_до_пересадки  \
1524                        041М            Ярославль-Главный   

      конечная точка_до_пересадки вид транспорта_до_пересадки  \
1524  Москва (Ярославский вокзал)                       train   

      отправления_до_пересадки     прибытие_до_пересадки  \
1524 2025-03-18 02:35:00+00:00 2025-03-18 06:43:00+00:00   

     начальная точка: Город Город пересадки  \
1524              Ярославль          Москва   

     номер маршрута_после_пересадки_df2 начальная точка_после_пересадки_df2  \
1524                               204В           Москва (Восточный вокзал)   

     конечная точка_после_пересадки_df2 вид транспорта_после_пересадки_df2  \
1524                              Тверь                              train   

     отправления_после_пересадки_df2 прибытие_после_пересадки_df2  \
1524       2025-03-18 08:56:00+00:00    2025-03-18 12:08:00+00:00   

     конечная точка: Город время пересадки  цена первого маршрута  

In [ ]:
pd.set_option('display.max_columns', None)
df_filtered

,номер маршрута_до_пересадки,начальная точка_до_пересадки,конечная точка_до_пересадки,вид транспорта_до_пересадки,отправления_до_пересадки,прибытие_до_пересадки,начальная точка: Город,Город пересадки,номер маршрута_после_пересадки_df2,начальная точка_после_пересадки_df2,конечная точка_после_пересадки_df2,вид транспорта_после_пересадки_df2,отправления_после_пересадки_df2,прибытие_после_пересадки_df2,конечная точка: Город,время пересадки,цена первого маршрута,цена второго маршрута,общая_цена,длительность первого маршрута,длительность второго маршрута,общее_время,рейтинг
1524,041М,Ярославль-Главный,Москва (Ярославский вокзал),train,2025-03-18 02:35:00+00:00,2025-03-18 06:43:00+00:00,Ярославль,Москва,204В,Москва (Восточный вокзал),Тверь,train,2025-03-18 08:56:00+00:00,2025-03-18 12:08:00+00:00,Тверь,2:13,1240,960,2200,4:08,3:12,9:33,1.0


Ранжирование прямых маршрутов (чем меньше рейтинг - тем более подходящий маршрут)

In [ ]:
import pandas as pd

def process_direct_routes(df, exclude_transport=None, preferred_time_range=None, weight_time=0.5, weight_price=0.5):
    """
    Обработка прямых маршрутов.

    df: DataFrame с маршрутами.
    exclude_transport: список типов транспорта, которые нужно исключить.
    preferred_time_range: кортеж (start_hour, end_hour) для фильтрации маршрутов по временному интервалу.
    weight_time: вес для приоритета времени в пути (0.0 - низкий, 1.0 - высокий).
    weight_price: вес для приоритета цены (0.0 - низкий, 1.0 - высокий).
    """
    # Удаляем дубликаты
    df = df.drop_duplicates(subset=['номер маршрута', 'начальная точка', 'конечная точка', 'вид транспорта', 'отправления', 'прибытие'])

    # Преобразуем "отправления" и "прибытие" в datetime
    df['отправления'] = pd.to_datetime(df['отправления'], errors='coerce')
    df['прибытие'] = pd.to_datetime(df['прибытие'], errors='coerce')

    # Исключаем указанные типы транспорта
    if exclude_transport:
        df = df[~df['вид транспорта'].isin(exclude_transport)]

    # Фильтруем по временному интервалу
    if preferred_time_range:
        start_hour, end_hour = preferred_time_range
        print(f"Фильтруем маршруты, которые ПОЛНОСТЬЮ находятся в интервале {start_hour}:00 - {end_hour}:00")

        if df[['отправления', 'прибытие']].isna().any().any():
            print("❌ Внимание! Есть пустые значения в 'отправления' или 'прибытие'. Удаляем их.")
            df = df.dropna(subset=['отправления', 'прибытие'])

        # Учет перехода через полночь
        if start_hour > end_hour:
            # Интервал пересекает 00:00 (например, 22:00 - 8:00)
            df = df[
                ((df['отправления'].dt.hour >= start_hour) | (df['отправления'].dt.hour < end_hour)) &
                ((df['прибытие'].dt.hour > start_hour) | (df['прибытие'].dt.hour <= end_hour))
            ]
        else:
            # Обычный дневной интервал (например, 7:00 - 21:00)
            df = df[
                (df['отправления'].dt.hour >= start_hour) &
                (df['прибытие'].dt.hour <= end_hour)
            ]

        print("Количество маршрутов после фильтрации:", df.shape[0])

    return df

# Тестируем функцию с разными интервалами
df_filtered_direct = process_direct_routes(direct_route, exclude_transport=['plane'], preferred_time_range=(22, 8))
print("\nРезультат:\n", df_filtered_direct)


Фильтруем маршруты, которые ПОЛНОСТЬЮ находятся в интервале 22:00 - 8:00
Количество маршрутов после фильтрации: 0

Результат:
 Empty DataFrame
Columns: [номер маршрута, начальная точка, конечная точка, вид транспорта, отправления, прибытие, время в секундах]
Index: []


In [ ]:
df_filtered_direct

,номер маршрута,начальная точка,конечная точка,вид транспорта,отправления,прибытие,время в секундах
